Подгрузим все необходимые нам библиотеки

In [ ]:
import pandas as pd
import numpy as np
!pip install pubchempy
!pip install requests
!pip install beautifulsoup4
import pubchempy as pbp
import re
import requests
from bs4 import BeautifulSoup 

Для начала попробуем извлечь Compound id для всех соединений. Для этого воспользуемся библиотекой Pubchempy.

In [ ]:
#Считываем изначальный датасет
data = pd.read_csv("7.csv")
#Забиваем в переменные колонки по которым будем искать CID
name = data['Name']
norm = data['normalised_name']
#Создаем пустой список в который будем вносить полученные значения
cid = []
#Запускаем цикл перебирающий все названия в столбце
for i in range(5000):
    #Поскольку в колонке name используются разные синонимы обозначающие одно и тоже стоит привести их к общему виду
    response = requests.get(url="https://en.wikipedia.org/wiki/" + name[i] )
    soup = BeautifulSoup(response.content, 'html.parser')
    #Если соединение есть в википедии то как правило в заголовке указано его тривиальное название, которое нам нужно
    title = soup.find(id='firstHeading')
    a = title.string
    #Записываем полученное из википедии название вместо изначального (Если соединения нет в википедии response возвращает значение без изменений)
    name.replace(to_replace = name[i], value = a)
    #Находим все синонимы соединения в библиотеке pubchempy
    synonyms = pbp.get_synonyms(name[i], 'name')
    syn = (' '.join(str(ol) for ol in synonyms))
    #Среди синонимов есть и CID номер. Ищем его используя регулярные выражения
    ci = re.findall("'CID': \d+", syn)
    st = (' '.join(str(j) for j in ci))
    #Отделяем от найденой строки цифры и записываем их в переменную
    vv = re.findall(r'\d+', st)
    #Проверим нашелся ли сид номер таким способом
    if vv == []:
        try:
            #Если по столбцу name CID номер мы не получили попробуем его найти из второго столбца содержащего smiles
            new = pbp.get_compounds(norm[i], 'smiles')  
            b = ((' '.join(str(el) for el in new)))
            #В результате мы получем "Compound123" из этого нам необходимы только цифровая часть
            v = re.findall(r'\d+', b)
            fin = int(''.join(v))    
            #Добавляем полученное цифровое значение в список
            cid.append(fin)
        except:
            #Если не получилось извлечь CID записываем NaN
            cid.append('NaN')
    else:
        fin = int(''.join(vv))
        #Если значение изнально нашлось среди синонимов тривиального названия оно добавляется в список
        cid.append(fin)

Полученные значения запишем в файл txt поскольку excel может автоматически отформатировать полученные значения при запуске файла

In [ ]:
MyFile = open ('output.txt', 'w')
for element in cid:
    MyFile.write(str(element))
    MyFile.write('\n')
MyFile.close()

Восстановим все данные DOI

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
!pip install crossref-commons
import crossref_commons.retrieval


In [ ]:
#Считываем файл содержащий исправленные DOI (в изначальном датасете некоторые DOI слиплись с названием журнала)
DOI = pd.read_csv('DOI.csv', encoding='latin-1')
#Удалим дубликаты для ускорения работы алгоритма
DO = DOI.drop_duplicates()
doi = DO['DOI']
doi = doi.reset_index(drop = True)

In [ ]:
print(DO.shape)

(2822, 1)


После удаления дубликатов осталось 2822 уникальных строк для которых мы будем искать дату публикации статьи, название журнала и название статьи.

In [ ]:
#Создаем пустые списки в которые будем записывать полученные данные
publish_date = []
artical_title = []
jornal_name = []
#Пройдемся по всем уникальным DOI
for i in range(2822):
  try:
    #С помощью библиотки извлечем данные публикации
    tmp = crossref_commons.retrieval.get_publication_as_json(doi[i])
    #В полученом словаре найдем дату выхода статьи
    date = tmp['indexed']['date-time']
    publ_date = date.split('T')[0]
    #После отделения времени от даты записываем полученную дату в список
    publish_date.append(publ_date)
    #Таким же спосоюом найдем название статьи
    titile = tmp['title']
    tit = (' '.join(str(ol) for ol in titile))
    #Записываем полученное значение 
    artical_title.append(tit)
    #И наконец найдем название журнала
    jornal= tmp['short-container-title']
    jorn = (' '.join(str(ol) for ol in jornal))
    #Его тоже записываем
    jornal_name.append(jorn)
  except:
    #Если статья не найдена добавляем во все списки 'Article not found'
    doi[i] = 'Article not found'
    publish_date.append('Article not found')
    artical_title.append('Article not found')
    jornal_name.append('Article not found')
#Создаем датафрейм из ролученных списков
DO['DOI'] = doi
DO['Date'] = publish_date
DO['Journal'] = jornal_name
DO['Title'] = artical_title
#Проверим выходной датафрейм
print(DO)

Полученные данные запишем в xlsx файл для дальнейшего использования

In [ ]:
DO.to_excel('DOI_new.xlsx', index = False)

In [ ]:
df = pd.DataFrame(pd.read_excel("7_new.xlsx"))
data = pd.DataFrame(pd.read_excel('DOI_new.xlsx'))

При помощи функции merge востанавливаваем убранные ранее дубликаты. Для это используем left.join а индексной колонкой делаем общую колонку 'DOI'

In [ ]:
dat = df['DOI']
fin = pd.merge(dat, data, how ='left', on = 'DOI')

Выгружаем полученный датасет и проверяем его

In [ ]:
fin.to_excel('fin.xlsx', index = False)

Поскольку в большинстве библиотек паристь удобнее по smiles найдем его

In [ ]:
import pandas as pd
!pip install pubchempy
import pubchempy as pbp

In [ ]:
#Считываем датасет
data = pd.read_excel('7_whit_cid.xlsx')
CID = data['CID']
#Удаляем дубликаты для ускорения работы алгоритма
cid = CID.drop_duplicates() 
ci = cid['CID'].tolist()

In [ ]:
#создадим пустой список в который будем записывать значения
canonical_smiles = []
for i in ci:
  try:
    #Находим smiles по CID которые ншли ранее
    c = pbp.Compound.from_cid(i)
    a = c.canonical_smiles
    canonical_smiles.append(a)
  except:
    #В случае ошибки вместо значения записываем NaN
    canonical_smiles.append('NaN')

Для правильного объединения изначального датасета и полученных smiles вставим колонку CID в датафрейм с дескрипторами

In [ ]:
canonical_smiles = canonical_smiles.insert(0, "CID", cid)

При помощи функции merge востанавливаваем убранные ранее дубликаты. Для это используем left.join а индексной колонкой делаем общую колонку 'CID'.
Полученный датафрейм выгружаем и провеяем.

In [ ]:
smiles = pd.merge(CID, canonical_smiles, how ='left', on = 'CID')
smiles.to_excel('smiles.xlsx', index = False)